In [290]:
import numpy as np
from numpy.typing import ArrayLike
from typing import Callable
from math import e, log, sqrt
from random import random
from scipy.stats import binom, chi2, norm

---
# Parcial 3 - Modelos y Simulación

## Ejercicio 2

In [70]:
# Datos del archivo ordenados
SAMPLES = np.sort([
    15.22860536, 40.60145536, 33.67482894,
    44.03841737, 15.69560109, 16.2321714,
    25.02174735, 30.34655637, 3.3181228,
    5.69447539, 10.1119561, 49.10266584,
    3.6536329, 35.82047148, 3.37816632,
    36.72299321, 50.67085322, 3.25476304,
    20.12426236, 20.2668814, 17.49593589,
    2.70768636, 14.77332745, 1.72267967,
    23.34685662, 8.46376635, 9.18330789,
    9.97428217, 2.33951729, 137.51657441,
    9.79485269, 10.40308179, 1.57849658,
    6.26959703, 4.74251574, 1.53479053,
    34.74136011, 27.47600572, 9.1075566,
    1.88056595, 27.59551348, 6.82283137,
    12.45162807, 28.01983651, 0.36890593,
    7.82520791, 3.17626161, 46.91791271,
    38.08371186, 41.10961135
])

#Número de muestras
NSAMPLES = len(SAMPLES)

In [169]:
LAMDA = 1/0.05
# fda exponencial
def Fexp(x:int):
    return 1 - e**(-x / LAMDA)

In [158]:
def KS_statistic(Nsamples:int, sort_samples:ArrayLike, F:Callable[[float], float]) -> float:
    """
    Calcula el estadístico de Kolmogorov-Smirnov

    Args:
        Nsamples (int): Número de muestras
        sort_samples (float): Lista ordenada de muestras

    Returns:
        float: estadístico
    """
    F_values = F(sort_samples)
    j = np.arange(1, Nsamples + 1)
    d_plus = np.max(j / Nsamples - F_values)
    d_minus = np.max(F_values - (j - 1) / Nsamples)
    return max(d_plus, d_minus)

In [159]:
d0 = KS_statistic(Nsamples=NSAMPLES, sort_samples=SAMPLES, F=Fexp)
print(f"🧐 D estadístico:{round(d0,4)}")

🧐 D estadístico:0.0744


---
c)
Determine si la hipotesis nula es rechazada o no, con un nivel de rechazo del 4%. Para esto, utilizar simulaciones con variables uniformes 

In [170]:
#Test de Kolomogorov-Smirnov
def KS_test(Nsim:int, Nsamples:int, d0:float,
            samples:ArrayLike, alpha:float):

    p_value = 0

    print(f"🧐 D estadístico:{round(d0,4)}")

    for _ in range(Nsim):
        #Generamos muestras aleatorias para estimar el p_valor
        uniform_samples = np.sort([random() for _ in range(Nsamples)])
        
        #Calculo el estadístico con max{j/Nsamples - U_j, U_j - (j-1) /Nsamples}
        d_sim = KS_statistic(Nsamples=Nsamples, sort_samples=uniform_samples, F=lambda x: x)

        if d_sim >= d0:
            p_value += 1
        
    p_value = p_value / Nsim
    print(f"☝️ p-valor obtenido: {p_value}")

    if p_value > alpha:
        print(f"😲☝️ Como {p_value} > {alpha}:")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"😲☝️ Como {p_value} <= {alpha}:")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {int(100 * (1-alpha))}%")

In [171]:
# Nivel de rechazo del 4% -> CONFIANZA 96%
ALPHA = 0.04
KS_test(Nsim=10_000, Nsamples=NSAMPLES, d0=d0, samples=SAMPLES, alpha=ALPHA)

🧐 D estadístico:0.0744
☝️ p-valor obtenido: 0.9284
😲☝️ Como 0.9284 > 0.04:
	 😒 No hay evidencia suficiente para rechazar Ho


---
d)  Determine si la hipotesis nula es rechazada o no, con un nivel de rechazo del 4%, esta vez simulando variables que verifiquen la hipotesis nula. 

In [172]:
def Xexp() -> float:
    """
    Variable aleatoria con distribución exponencial
    con LAMBDA = 0.04

    Returns:
        float: _description_
    """
    U = 1 - random()
    return -log(U) / LAMDA

In [177]:
#Test de Kolomogorov-Smirnov
def KS_test(Nsim:int, Nsamples:int, d0:float, alpha:float):

    p_value = 0

    print(f"🧐 D estadístico:{round(d0, 4)}")

    for _ in range(Nsim):
        #Generamos muestras aleatorias para estimar el p_valor
        uniform_samples = np.sort([Xexp() for _ in range(Nsamples)])
        d_sim = KS_statistic(Nsamples=Nsamples, sort_samples=uniform_samples, F=lambda x: x)
        
        if d_sim >= d0: 
            p_value += 1
        
    p_value = p_value / Nsim
    print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

    if p_value > alpha:
        print(f"😲☝️ Como {p_value} > {alpha}:")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"😲☝️ Como {p_value} <= {alpha}:")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {int(100 * (1-alpha))}%")

In [178]:
KS_test(Nsim=10_000, Nsamples=NSAMPLES, d0=d0, alpha=ALPHA)

🧐 D estadístico:0.0744
☝️ p-valor obtenido: 1.0
😲☝️ Como 1.0 > 0.04:
	 😒 No hay evidencia suficiente para rechazar Ho


El $p_{valor}$ es 1 pues todos los $d_{sim}$ calculados son mayores que el estadístico $d_0$

---
# Ejercicio 3


In [ ]:
OBSERVED_FREQUENCIES = [
    38, 144, 342,
    287, 164, 25
]

n = len(OBSERVED_FREQUENCIES)

NSAMPLES = 1000

Ns = [0, 1, 2, 3, 4, 5]

In [232]:
# Estimación de p con la media muestral
MEAN = sum(Ni * Xi for Ni, Xi in zip(Ns, OBSERVED_FREQUENCIES)) / NSAMPLES

P_HAT = MEAN / n 

P_HAT

0.4116666666666667

In [233]:
# Calculamos los valores de las probabilidades
PROBABILITY_VALUES = [binom.pmf(k=i, n=n ,p=P_HAT) for i in range(n)]
PROBABILITY_VALUES

[np.float64(0.04147063926608641),
 np.float64(0.1741061965788672),
 np.float64(0.3045625393412195),
 np.float64(0.28414333226546257),
 np.float64(0.14911487904299417),
 np.float64(0.04173526926189185)]

In [267]:
def Pearson_statistic(Nsamples:int, samples:list[int], probability_values:list[float]) -> float:
    statistic = 0
    for sample, prob_i in zip(samples, probability_values):
        statistic += (sample - Nsamples * prob_i) ** 2 / (Nsamples * prob_i)
    return statistic

def Pearson_test(Nsamples: int, t0:float, alpha:float):

    df = Nsamples - 1 - 1

    p_value = round(1 - chi2.cdf(round(t0, 4), df=df), 4)

    print(f"☝️ p-valor obtenido: {p_value}")

    if p_value > alpha:
        print(f"☝️😲 Cómo {p_value} > {alpha}")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"☝️😲 Cómo {p_value} <= {alpha}")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {100 * (1-alpha)}%")

In [271]:
t0 = Pearson_statistic(Nsamples=NSAMPLES, samples=OBSERVED_FREQUENCIES,
                       probability_values=PROBABILITY_VALUES)
print(f"🧐 T estadístico:{round(t0, 4)}")

🧐 T estadístico:18.3235


In [272]:
ALPHA = 0.05
Pearson_test(
        Nsamples=n,
        t0=t0,
        alpha=ALPHA
)

☝️ p-valor obtenido: 0.0011
☝️😲 Cómo 0.0011 <= 0.05
	 🔴 Se rechaza Ho con una confianza del 95.0%


In [303]:
def XBin(n: int, p: float) -> int:
    """
    Variable aleatoria Binomial

    Args:
        n (int): ensayos
        p (float): probabilidad de éxito

    Returns:
        tuple: número de éxitos alcanzados
    """
    success = 0
    iterations = 0
    for _ in range(n):
        if random() < p:
            success += 1
    return success

In [307]:
def p_value_estimation(Nsim:int, Nsamples:int, n:int, t0:float):
    p_value = 0

    for _ in range(Nsim):
        mean = 0
        p_hat = 0

        #Generamos Nsamples muestras de las posibles frecuencias observadas,
        #cada una con probabilidad p_i
        samples = [XBin(n=n, p=P_HAT) for _ in range(Nsamples)]
        
        #Cuento la cantidad de frecuencias observadas
        N_i = np.bincount(samples, minlength=n+1)

        #Calculo las probabilidades
        probability_values = [binom.pmf(k=i, n=n ,p=P_HAT) for i in range(n)]
        
        
        T = Pearson_statistic(Nsamples=Nsamples, samples=N_i, probability_values=probability_values)

        if T > t0:
            p_value += 1

    return p_value / Nsim

In [312]:
p_value = p_value_estimation(
            Nsim=1_000,
            Nsamples=NSAMPLES,
            n=n,
            t0=t0
            )

if p_value > ALPHA:
    print(f"☝️😲 Cómo {p_value} > {ALPHA}")
    print("\t 😒 No hay evidencia suficiente para rechazar Ho")
else:
    print(f"☝️😲 Cómo {p_value} <= {ALPHA}")
    print(f"\t 🔴 Se rechaza Ho con una confianza del {100 * (1-ALPHA)}%")

☝️😲 Cómo 0.002 <= 0.05
	 🔴 Se rechaza Ho con una confianza del 95.0%


---
# Ejercicio 4

Estimar mediante el metodo de Monte Carlo el valor de la siguiente integral:

$$
\int_2³ e^{-x}\cdot(1-x⁴)dx
$$

In [299]:
def g(x:int) -> float:
    """
    Función dada por el ejercicio I

    Args:
        x (int): Entradas

    Returns:
        float: Resultado tras evaluar g en x
    """
    return (e ** -x) * (1 - x ** 4) 

def MonteCarlo_map(x:float) -> float:
    """
    Mapeo de MonteCarlo

    Args:
        x (float): parámetro de entrada

    Returns:
        float: Mapeo de x evaluado en g hacia el [0,1]
    """
    return g(2 + (3-2) * x) * (3-2)

In [300]:
def integral_estimation(z_alpha_2:float, precision:float) -> list[tuple[int, float, float, float]]:
    """
    Estimación de la integral por el método de Monte Carlo
    - Se tiene en cuenta el criterio de parada

    Args:
        z_alpha_2 (float): Valor de la tabla de la Normal para alpha / 2
        precision (float): Según el ejercicio 0.001

    Returns:
        list[tuple[int, float, float, float]]: Tupla con
        - Número de iteraciones
        - Estimación de Monte Carlo
        - Intervalo de Confianza
        - Desvío Estándar
    """
    results = []
    mean = MonteCarlo_map(x=random())
    Scuad, n = 0, 1
    while not (n > 100 and sqrt(Scuad/n) < precision):
        n += 1
        g_U = MonteCarlo_map(x=random())
        prev_mean = mean
        mean = prev_mean + (g_U - prev_mean) / n
        Scuad = Scuad * (1 - 1/(n-1)) + n * (mean - prev_mean) ** 2
        if n == 1000 or n == 5000 or n == 7000:
            I = (mean - z_alpha_2 * sqrt(Scuad / n), mean + z_alpha_2 * sqrt(Scuad / n)) 
            I_Long = I[1] - I[0] 
            S = sqrt(Scuad / n)
            results.append((n, mean, I, I_Long, S))
    I = (mean - z_alpha_2 * sqrt(Scuad / n), mean + z_alpha_2 * sqrt(Scuad / n)) 
    I_Long = I[1] - I[0] 
    S = sqrt(Scuad / n)
    results.append((n, mean, I, I_Long, S))
    return results

In [301]:
def pretty_print_results(results: list[tuple[int, float, tuple[float, float],float, float]]) -> None:
    print("="*100)
    print(f"{'Iteraciones':>12} | {'Estimación':>12} | {'Intervalo Confianza':^24} | {'Longitud I ':>10}| {'Desvío':>10}")
    print("-"*100)
    for n, mean, (lower, upper), I_Long, std in results:
        print(f"{n:12} | {mean:12.4f} | [{lower:10.4f}, {upper:10.4f}] | {I_Long:10.4f} |  {std:10.4f}")
    print("="*100)

In [302]:
L = 0.001 * 2
confiance = 0.95
alpha = 1 - confiance
z_alpha_2 = abs(norm.ppf(alpha/2))
precision = L / (2 * z_alpha_2)
results = integral_estimation(z_alpha_2=z_alpha_2, precision=precision)
pretty_print_results(results=results)

 Iteraciones |   Estimación |   Intervalo Confianza    | Longitud I |     Desvío
----------------------------------------------------------------------------------------------------
        1000 |      -3.0851 | [   -3.1205,    -3.0496] |     0.0709 |      0.0181
        5000 |      -3.1019 | [   -3.1176,    -3.0862] |     0.0313 |      0.0080
        7000 |      -3.0963 | [   -3.1096,    -3.0830] |     0.0267 |      0.0068
     1251047 |      -3.0845 | [   -3.0855,    -3.0835] |     0.0020 |      0.0005


El valor de la última fila de la primer columna de la tabla indica el valor $N_s$

A su vez según lo pedido en el ejercicio:
- Estimación -> $\bar{I}$
- Desvío -> S
- Intervalo de Confianza -> IC(95 %)